In [ ]:
%pip install git+https://github.com/Pedro-HFelix/IA.git

625.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


zsh:1: command not found: pip


In [17]:
from trees_classifiers.ID3 import DecisionTreeID3
from trees_classifiers.C45 import DecisionTreeC45
from trees_classifiers.CART import DecisionTreeCART
import pandas as pd

In [20]:
data = pd.read_csv('Test.csv')
features = ['outlook', 'temperature', 'humidity', 'windy']
target = 'play'

print("--- Algoritmo CART (com dados diretos do CSV) ---")
model_cart = DecisionTreeCART(data=data, features=features, target_class=target)
model_cart.build()
print("Árvore Gerada:")
print(model_cart.tree)
print("-" * 30)

print("--- Exemplo de Previsão (CART) ---")
example = {'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'high', 'windy': True}
prediction = model_cart.predict(example)
print(f"Para o exemplo {example}, a previsão é: '{prediction}'")

--- Algoritmo CART (com dados diretos do CSV) ---
Árvore Gerada:
{'outlook': {'== overcast': 'yes', '!= overcast': {'humidity': {'== high': {'outlook': {'== sunny': 'no', '!= sunny': {'windy': {'<= 0.5': 'yes', '> 0.5': 'no'}}}}, '!= high': {'windy': {'<= 0.5': 'yes', '> 0.5': {'outlook': {'== rainy': 'no', '!= rainy': 'yes'}}}}}}}}
------------------------------
--- Exemplo de Previsão (CART) ---


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/trees_classifiers/CART.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[feature].fillna(mode_val, inplace=True)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/trees_classifiers/CART.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the int

IndexError: list index out of range